In [ ]:
import scipy.io as sio
import pandas as pd
from sklearn.metrics import roc_curve, roc_auc_score, auc
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.utils import resample
from sklearn.model_selection import LeaveOneOut, GridSearchCV
import numpy as np
import plotly.graph_objects as go
import xgboost as xgb

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
def ml_classifier(input_table, target, n_iterations, classifier_type='xgb'):
    """
    Train a classifier (XGBoost, Lasso, ElasticNet, Ridge) using Leave-One-Out cross-validation and obtain test-set results
    via bootstrap.

    Args:
        input_table (pd.DataFrame): Input dataset containing features and class labels.
        target (string): Column to predict.
        n_iterations (int): Number of bootstrap iterations to perform.
        classifier_type (string): Type of classifier to use ('xgb', 'lasso', 'elasticnet', 'ridge').

    Returns:
        dict: Dictionary containing the AUC, FPR, TPR, thresholds, and feature importances for each iteration.

    Classifier Details:
        'xgb': XGBoost is a gradient boosting algorithm.
               Advantages: Handles missing values, non-linear data, and performs well in many scenarios.
               Disadvantages: Might overfit on noisy data, requires careful tuning.

        'lasso': Lasso classification uses Logistic Regression with L1 penalty.
                 Advantages: Performs feature selection by pushing less important features' coefficients to zero.
                 Disadvantages: Can't select more features than samples.

        'elasticnet': ElasticNet classification uses a combination of L1 and L2 penalties.
                      Advantages: Can learn a sparse model where few features are influential, and can select more features than samples.
                      Disadvantages: Requires setting two hyperparameters.

        'ridge': Ridge classification uses Logistic Regression with L2 penalty.
                 Advantages: Prevents multicollinearity in regression model.
                 Disadvantages: Doesn't perform feature selection like Lasso.

    """
    # Bootstrap 90% of the sample size each time
    np.random.seed(42)
    n_size = int(len(input_table) * 0.9)

    # Initialize variables
    stats = list()
    metrics = ['auc', 'fpr', 'tpr', 'thresholds', 'feature_importances']
    results = {'main': {m: [] for m in metrics}}

    # Define hyperparameter grid for C
    param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
    
    for i in tqdm(range(n_iterations), desc="Bootstrap iterations"):
        subsampled_data = resample(input_table, n_samples=n_size, stratify=input_table[target].values)
        y = subsampled_data[target].values
        X = subsampled_data.drop(columns=[target]).values

        # Data normalization
        scaler = StandardScaler()
        X = scaler.fit_transform(X)

        loo = LeaveOneOut()
        loo.get_n_splits(X)

        labels = []
        probabilities = []
        coefficients = []

        for train_index, test_index in loo.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            if classifier_type == 'xgb':
                clf = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
                clf.fit(X_train, y_train)
                coefficients.append(clf.feature_importances_)

            elif classifier_type in ['lasso', 'elasticnet', 'ridge']:
                if classifier_type == 'lasso':
                    base_clf = LogisticRegression(penalty='l1', solver='liblinear', class_weight='balanced', max_iter=10000, tol=1e-2)
                if classifier_type == 'elasticnet':
                    base_clf = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, class_weight='balanced', max_iter=10000, tol=1e-2)
                elif classifier_type == 'ridge':
                    base_clf = LogisticRegression(penalty='l2', solver='sag', class_weight='balanced', max_iter=10000, tol=1e-2)

                # Apply grid search on the base classifier
                grid_clf = GridSearchCV(base_clf, param_grid, cv=3)  # Using 3-fold CV for grid search
                grid_clf.fit(X_train, y_train)
                clf = grid_clf.best_estimator_  # Get the best classifier
                coefficients.append(np.abs(clf.coef_[0]))

            labels.append(y_test)
            probabilities.append(clf.predict_proba(X_test)[:, 1])

        stats.append(roc_auc_score(labels, probabilities))
        fpr, tpr, thresholds = roc_curve(labels, probabilities)
        results['main']['fpr'].append(fpr)
        results['main']['tpr'].append(tpr)
        results['main']['thresholds'].append(thresholds)
        results['main']['auc'].append(roc_auc_score(labels, probabilities))
        results['main']['feature_importances'].append(np.mean(coefficients, axis=0))

    return results

def plot_roc_all_features(results, n_iterations):
    """
    This function creates and plots a ROC for a classifier's performance using all features.

    Args:
    results (dict): A dictionary containing the results of the classifier, including 'fpr', 'tpr', and 'auc' keys.
    n_iterations (int): The number of iterations used for creating the interpolated TPR values.

    Returns:
    None: The function saves the ROC plot as an image file and displays it.
    """
    # Set plot parameters
    colors = {
        'filla': 'rgba(52, 152, 219, 0.2)',
        'linea': 'rgba(52, 152, 219, 0.5)',
        'maina': 'rgba(41, 128, 185, 1.0)',
        'grid': 'rgba(189, 195, 199, 0.5)',
        'annot': 'rgba(149, 165, 166, 0.5)',
        'highlight': 'rgba(192, 57, 43, 1.0)'
    }

    fpr_mean = np.linspace(0, 1, n_iterations)
    interp_tprs = []

    # Calculate confidence bands
    for i in range(n_iterations):
        fpr, tpr = results['main']['fpr'][i], results['main']['tpr'][i]
        interp_tprs.append(np.interp(fpr_mean, fpr, tpr))
        interp_tprs[-1][0] = 0.0

    tpr_mean = np.mean(interp_tprs, axis=0)
    tpr_mean[-1] = 1.0

    tpr_ci = np.std(interp_tprs, axis=0) * 1.96
    tpr_upper = np.clip(tpr_mean + tpr_ci, 0, 1)
    tpr_lower = tpr_mean - tpr_ci

    auc = np.mean(results['main']['auc'])

    plot_data = [
        go.Scatter(x=fpr_mean, y=tpr_upper, line=dict(color=colors['linea'], width=1), hoverinfo="skip", showlegend=False, name='upper'),
        go.Scatter(x=fpr_mean, y=tpr_lower, fill='tonexty', fillcolor=colors['filla'], line=dict(color=colors['linea'], width=1), hoverinfo="skip", showlegend=False, name='lower'),
        go.Scatter(x=fpr_mean, y=tpr_mean, line=dict(color=colors['maina'], width=2), hoverinfo="skip", showlegend=True, name=f'AUC = {auc:.3f} [{tpr_lower.mean():.3f} {tpr_upper.mean():.3f}]')
    ]

    fig = go.Figure(plot_data)
    fig.add_shape(type='line', line=dict(dash='dash'), x0=0, x1=1, y0=0, y1=1)

    fig.update_layout(
        template='plotly_white',
        title_x=0.5,
        xaxis_title="1 - Specificity",
        yaxis_title="Sensitivity",
        width=600,
        height=600,
        legend=dict(yanchor="bottom", xanchor="right", x=0.95, y=0.01),
        font=dict(family="Arial", size=22, color="black")
    )

    fig.update_yaxes(range=[0, 1], gridcolor=colors['grid'], scaleanchor="x", scaleratio=1, linecolor='black')
    fig.update_xaxes(range=[0, 1], gridcolor=colors['grid'], constrain='domain', linecolor='black')

    fig.show()

def plot_feature_importances(results, input_table, target):
    """
    Plot the feature importances based on the output of ml_classifier using a horizontal bar plot.

    Args:
        results (dict): The results dictionary from ml_classifier.
        input_table (pd.DataFrame): Input dataset.
        target (string): Target column.

    Returns:
        None: Shows the feature importance plot.
    """

    # Correctly access 'feature_importances' from the nested dictionary structure
    feature_importance_mean = np.mean(results['main']['feature_importances'], axis=0)
    feature_importance_std = np.std(results['main']['feature_importances'], axis=0)

    # Calculate SEM
    n = len(results['main']['feature_importances'])
    sem = feature_importance_std / np.sqrt(n)

    # Get feature names
    feature_names = input_table.drop(columns=[target]).columns.tolist()

    # Match feature name convention from the group classifier
    def rearrange_name(name):
        parts = name.split()
        type_ = parts[1]   # 'wPLI', 'AEC-c'
        feature = parts[2] # 'N2-delta', 'N2-theta'
        return f"{feature}-{type_}"

    # Rearrange names
    feature_names = [rearrange_name(name) for name in feature_names]
    
    # Sorting the features by importance in ascending order
    sorted_idx = feature_importance_mean.argsort()
    feature_importance_mean = feature_importance_mean[sorted_idx]
    sem = sem[sorted_idx]
    feature_names_sorted = [feature_names[i] for i in sorted_idx]
    
    # Select the top ten features
    top_n = 10
    feature_importance_mean = feature_importance_mean[-top_n:]
    sem = sem[-top_n:]
    feature_names_sorted = feature_names_sorted[-top_n:]

    # Create a horizontal bar plot
    fig = go.Figure()

    # We use y-axis for feature names and x-axis for feature importance values in horizontal bar plots
    fig.add_trace(
        go.Bar(
            y=feature_names_sorted,
            x=feature_importance_mean,
            orientation='h',
            error_x=dict(type='data', array=sem, visible=True)
        )
    )

    # Layout configuration
    fig.update_layout(
        # title="Feature Importances with SEM",
        xaxis_title="Feature Importance",
        yaxis_title="GE features",
        template='plotly_white',
        width=600,
        height=600,
        font=dict(family="Arial", size=22, color="black")
    )

    fig.show()

In [ ]:
### Load MoCA decline data
decline_df = pd.read_excel('./MoCA_change_2.xlsx')
decline_df

In [ ]:
# Remove ID duplicates and unnecessary columns
decline_df = decline_df.drop_duplicates(subset='ID', keep='first')
decline_df = decline_df.drop(columns = ['Sbj_ID_Date', 'Categorization',	'MoCA',	'MoCA_DurationFollowup_Years',	'MoCA_Time_Coded_FirstLast',	'MoCA_Score_FirstVisit',	'MoCA_Score_LastVisit',	'MoCA_AnnualChange'], axis = 1)
decline_df

In [ ]:
## Number of subjects to analyze
len(decline_df)

In [ ]:
## Load all Global Efficiency features
all_features = pd.read_csv('./all_GE_AEC_wPLI_features.csv')
## Remove subjects not having all the stages (NaNs)
all_features = all_features.dropna()
all_features

In [ ]:
## Merge the MoCA decline tertiles with the GE features
df = pd.merge(decline_df, all_features, on = 'ID')
df = df.drop(columns = ['ID', 'Class'], axis = 1)
df

In [ ]:
# Leave most affected class (1st tertile) as class number 1, and change the rest to class 0
df['MoCA_AnnualChange_Tertile'] = df['MoCA_AnnualChange_Tertile'].replace({2: 0, 3: 0})
df

In [ ]:
# Run the XGBoost classifier
# Number of iterations for bootstrap
n_iterations = 1000

results = ml_classifier(df, "MoCA_AnnualChange_Tertile", n_iterations, classifier_type='xgb')

In [ ]:
plot_roc_all_features(results, n_iterations)

In [ ]:
plot_feature_importances(results, df, "MoCA_AnnualChange_Tertile")